# FruitScan - Preprocesamiento del Dataset
Este notebook realiza el preprocesamiento de imágenes del dataset `Fruit Quality` para usar en un modelo de detección de defectos con PyTorch.

In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
# 1. Configuración inicial
DATA_DIR = "./data/FruitQuality"  # Ruta al dataset descargado y organizado
BATCH_SIZE = 8
IMG_SIZE = 224
VAL_SPLIT = 0.2
SEED = 42


In [ ]:
# 2. Transformaciones (aumentos opcionales)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Imagen RGB
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
# 3. Dataset a partir de carpetas
dataset = datasets.ImageFolder(root=DATA_DIR, transform=transform)
class_names = dataset.classes

# 4. Dividir en entrenamiento y validación
torch.manual_seed(SEED)
val_size = int(len(dataset) * VAL_SPLIT)
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [ ]:
# 5. DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


In [ ]:
# 6. Visualización rápida
def show_sample(loader):
    images, labels = next(iter(loader))
    fig, ax = plt.subplots(1, 4, figsize=(12, 3))
    for i in range(4):
        img = images[i].permute(1, 2, 0)  # CHW -> HWC
        img = img * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])
        ax[i].imshow(img)
        ax[i].set_title(class_names[labels[i]])
        ax[i].axis("off")
    plt.show()

show_sample(train_loader)


In [ ]:
# 7. Guardar información útil
torch.save({
    'class_names': class_names,
    'transform': transform
}, 'data/fruitscan_meta.pth')

print("✅ Dataset preprocesado y listo para entrenamiento.")
